In [1]:
import os
import sys

current_dir = os.getcwd()

parent_dir = os.path.abspath(os.path.join(current_dir, ".."))

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [2]:
from treemort.utils.config import setup

config_file_path = "../configs/sa_unet_bs8_cs256.txt"
conf = setup(config_file_path)

# Modified Config Variables for Local Execution; comment on HPC
conf.data_folder = "/Users/anisr/Documents/AerialImages"
conf.output_dir = os.path.join("..", conf.output_dir)

print(conf)

Namespace(data_folder='/Users/anisr/Documents/AerialImages', hdf5_file='AerialImageModel_ITD.h5', model='sa_unet', backbone=None, epochs=100, train_batch_size=8, val_batch_size=8, test_batch_size=8, train_crop_size=256, val_crop_size=256, test_crop_size=256, val_size=0.2, test_size=0.1, input_channels=4, output_channels=1, output_dir='../output/sa_unet', model_weights='best', learning_rate=0.0001, threshold=0.5, activation='tanh', loss='hybrid', resume=True)


In [3]:
from treemort.main import run

eval_only = False

run(conf, eval_only)

/Users/anisr/Documents/TreeSeg/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import h5py

from treemort.data.loader import prepare_datasets
from treemort.utils.config import setup
from treemort.utils.plots import plot_examples

config_file_path = "../configs/unet_bs8_cs256.txt"
conf = setup(config_file_path)

# Modified Config Variables for Local Execution; comment on HPC
conf.data_folder = "/Users/anisr/Documents/AerialImages"

hdf5_file_path = os.path.join(conf.data_folder, conf.hdf5_file)
with h5py.File(hdf5_file_path, 'r') as hf:
    keys = list(hf.keys())

train_dataset, val_dataset, test_dataset = prepare_datasets(conf)

plot_examples(train_dataset, num_examples=5)